In [1]:
!pip install pyflyt
!pip uninstall stable-baselines3 sb3_contrib -y
!pip install stable-baselines3 sb3_contrib
# !pip install stable-baselines3[extra]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 4.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 16.8 MB/s eta 0:00:0000:0100:01
Found existing installation: stable-baselines3 2.1.0
Uninstalling stable-baselines3-2.1.0:
  Successfully uninstalled stable-baselines3-2.1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires stable-baselines3==2.1.0, but you have stable-baselines3 2.3.2 which is incompatible.


In [2]:
# !rm -r *

In [2]:
import gymnasium as gym
from sb3_contrib import TQC
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
from stable_baselines3.common.callbacks import BaseCallback

import numpy as np
import time
from PyFlyt.core.drones import Rocket
import PyFlyt.gym_envs
import torch.nn as nn
import os
from torch.utils.tensorboard import SummaryWriter

2024-05-18 22:31:32.523025: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 22:31:32.523121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 22:31:32.664344: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
pybullet build time: Nov 28 2023 23:45:17


In [3]:
# Function to create a monitored environment
def make_env():
    def _init():
        env = gym.make("PyFlyt/Rocket-Landing-v1")
        return Monitor(env, "./monitor_train_logs")
    return _init

class AggregateRewardLoggingCallback(BaseCallback):
    def __init__(self, log_dir='custom_tensorboard/',verbose=0, check_freq=1000):
        super(AggregateRewardLoggingCallback, self).__init__(verbose)
        self.best_mean_reward = -np.inf
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.writer = SummaryWriter(log_dir=self.log_dir)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            all_rewards = []
            # Loop through all wrapped environments
            for env in self.training_env.envs:
                # Check and collect rewards from each environment
                if hasattr(env, 'get_episode_rewards'):
                    episode_rewards = env.get_episode_rewards()  # Use if method is available
                else:
                    episode_rewards = env.episode_rewards  # Direct attribute access if method is not available
                all_rewards.extend(episode_rewards)  # Collecting rewards from all environments

            # Compute overall mean reward across all environments
            mean_reward = np.mean(all_rewards)
            self.writer.add_scalar("Mean Episode Reward", mean_reward, self.num_timesteps)
            if mean_reward > self.best_mean_reward:
                self.best_mean_reward = mean_reward
                print(f"New best mean reward across all envs: {self.best_mean_reward}")
        return True
    
    def _on_training_end(self):
        self.writer.close()

In [7]:
# # Set up multiple environments for SubprocVecEnv
# n_envs = 700
# train_env = DummyVecEnv([make_env() for _ in range(n_envs)])

# eval_env = DummyVecEnv([make_env()])
# eval_callback = EvalCallback(eval_env, best_model_save_path="./logs/", eval_freq=500)
# checkpoint_callback = CheckpointCallback(save_freq=1000, save_path="./checkpoints/")


In [4]:
train_env = DummyVecEnv([make_env()])
checkpoint_callback = CheckpointCallback(save_freq=1000, save_path="./checkpoints/")
reward_callback = AggregateRewardLoggingCallback(log_dir='custom_tensorboard/',
                                                verbose=1)

In [5]:
# Policy settings for TQC
policy_kwargs = {
    "net_arch": [256, 256, 128],  # Architecture for the policy network
    "activation_fn": nn.ReLU,  # Properly reference activation functions
}

tqc_params = {
    "batch_size": 256,  # Increased batch size
    "learning_rate": 0.0003,
    "gradient_steps": 64,
    "train_freq": 64,
}

In [ ]:
# int(1e6), int(3e6)

In [6]:
model = TQC("MlpPolicy", train_env,
            policy_kwargs=policy_kwargs, verbose=1, device='cuda',
           tensorboard_log="./tensorboard_logs/", **tqc_params)
model.learn(total_timesteps=int(1e6),
            callback=[checkpoint_callback, reward_callback])

Using cuda device
                             
Logging to ./tensorboard_logs/TQC_1
argv[0]=
                             
argv[0]=
                             
argv[0]=
                             
argv[0]=
                             
argv[0]=
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 141       |
|    ep_rew_mean     | -2.37e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 41        |
|    time_elapsed    | 13        |
|    total_timesteps | 565       |
| train/             |           |
|    actor_loss      | 62.8      |
|    critic_loss     | 3.09      |
|    ent_coef        | 0.982     |
|    ent_coef_loss   | -0.0167   |
|    learning_rate   | 0.0003    |
|    n_updates       | 448       |
----------------------------------
                             
argv[0]=
                             
argv[0]=
New best mean reward across all envs: -2298.7283231033084
                    

In [ ]:
model.save("TQC_rocket_landing_v9_manual")

In [ ]:
int(3e6), int(1e7)

In [3]:
def get_last_model(check_point_path="/kaggle/working/checkpoints"):
    import os
    return os.path.join(check_point_path, sorted(os.listdir(check_point_path),
       key=lambda x : int(x.split("_")[2]),
      reverse=True)[0])
def copy_last_model(check_point_path="/kaggle/working/checkpoints", dst="/kaggle/working/"):
    import shutil
    src = get_last_model(check_point_path)
    shutil.copy(src, dst)
    print(f"Copied the model {src.split('/')[-1]}")

In [4]:
copy_last_model()

Copied the model rl_model_1000000_steps.zip


In [1]:
import os
import shutil
from pathlib import Path
 
def copy_latest_file(src_dir, dest_dir):
    # Get list of files in the source directory
    files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]
    if not files:
        print("No files found in the source directory.")
        return
 
    # Find the file with the latest timestamp
    latest_file = max(files, key=lambda x: os.path.getmtime(os.path.join(src_dir, x)))
    # Construct full file paths
    src_file = os.path.join(src_dir, latest_file)
    dest_file = os.path.join(dest_dir, latest_file)
    # Copy the latest file to the destination directory
    shutil.copy2(src_file, dest_file)
    print(f"Copied {latest_file} to {dest_dir}")
 


In [2]:
# Example usage
source_directory = '/kaggle/working/checkpoints'
destination_directory = '/kaggle/working'
copy_latest_file(source_directory, destination_directory)

Copied rl_model_999000_steps.zip to /kaggle/working


In [8]:
os.path.getmtime('/kaggle/working/checkpoints/rl_model_999000_steps.zip'), os.path.getmtime('/kaggle/working/checkpoints/rl_model_1000000_steps.zip')

(1716108390.8583019, 1716108347.5085618)

In [9]:

os.path.getmtime('/kaggle/working/checkpoints/rl_model_838000_steps.zip')

1716108383.3486536

In [5]:
source_directory, sorted(os.listdir(source_directory),
       key=lambda x : int(x.split("_")[2]),
      reverse=True)

('/kaggle/working/checkpoints',
 ['rl_model_1000000_steps.zip',
  'rl_model_999000_steps.zip',
  'rl_model_998000_steps.zip',
  'rl_model_997000_steps.zip',
  'rl_model_996000_steps.zip',
  'rl_model_995000_steps.zip',
  'rl_model_994000_steps.zip',
  'rl_model_993000_steps.zip',
  'rl_model_992000_steps.zip',
  'rl_model_991000_steps.zip',
  'rl_model_990000_steps.zip',
  'rl_model_989000_steps.zip',
  'rl_model_988000_steps.zip',
  'rl_model_987000_steps.zip',
  'rl_model_986000_steps.zip',
  'rl_model_985000_steps.zip',
  'rl_model_984000_steps.zip',
  'rl_model_983000_steps.zip',
  'rl_model_982000_steps.zip',
  'rl_model_981000_steps.zip',
  'rl_model_980000_steps.zip',
  'rl_model_979000_steps.zip',
  'rl_model_978000_steps.zip',
  'rl_model_977000_steps.zip',
  'rl_model_976000_steps.zip',
  'rl_model_975000_steps.zip',
  'rl_model_974000_steps.zip',
  'rl_model_973000_steps.zip',
  'rl_model_972000_steps.zip',
  'rl_model_971000_steps.zip',
  'rl_model_970000_steps.zip',
  'rl_